<a href="https://colab.research.google.com/github/ciupava/Yolo_experiments/blob/main/YoloTraining_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Training** ###

TO BE EDITED

(will get back to this once the preprocessing is fixed)

In this phase, we will not have all the data versions listed below used from Omdena, but only the datasets previously used in fAIr with RAMP.

TODO:

- [ ] understand where metric is calculated
- [ ] add the other metrics ... ??
- [ ] understand which checkpoint is saved/early stopping/metric
- [ ] access all the city folders, one by one, and run the ML thing
- [ ] save the results for each city and metric, to be able to compare



---

This notebook trains the YOLOv8 segmentation model to detect a new rooftop class. We considered the available YOLO configurations—nano, small, medium, large, and xl. While the larger sizes generally yield better results, they come with higher computational demands. To balance performance and computational expense, we chose the small YOLO model.

The training datasets were derived from the RAMP dataset and prepared using the preprocessing notebook. To optimize the model results, we trained YOLO using data version 1, 2, and 3. Each data version was successively pruned (V3, V2, .. V0), meaning V3 was pruned from V2 and so on to V0 so that each version contains the prunings from all of the previous.

RAMP-DATA-V0: 'Full Dataset' is 100k image-label pairs converted from RAMP's TIF-GEOJSON formats within 22 regional folders and stored in train-val-test folders with a 70-15-15 split; zipped at 15.2 GB downloaded from Source Cooperative.

YOLO-DATA-V1: 'Preprocessing' was a process that resulted in 83k image-label pairs in JPG-TXT formats from 20 regions, zipped at 3.5 GB. We pruned 17k images from V0, reducing the background images from 18% to 5%. Shanghai and Paris regions were removed due to image format issues. A minimal manual review discovered a few labeling issues (n=38), ensuring the highest data quality.

YOLO-DATA-V2: 'Pruning #2' resulted in 79k image-label pairs in JPG-TXT formats from 20 regions, zipped at 3.3 GB. We scored and analyzed inference results from the YOLO model trained on DATA-V1. The analysis revealed double the expected number of left-tail outliers. To improve labeling accuracy, we removed 4k outliers, assuming that most of these labels were errors.

YOLO-DATA-V3: 'Zoom 19' resulted in 55k image-label pairs in JPG-TXT formats from 20 regions, zipped at 2.4 GB. We extracted the x and y scales from the TIF files and found the range to be between 0.3 and 0.52 meters per pixel, with 70% between 0.3 and 0.4. To test the effect of training near Zoom 19, which equates to 0.3 meters per pixel, we removed all image labels with scales greater than 0.4.

To relieve the time constraints, we recommend training with a GPU. Google Colab's High-RAM T4 can train 100 epochs in 15 hours using a large batch size and cache enabled to minimize disk I/O. Using a 2023 laptop with a CPU, training YOLO for 100 epochs takes about 2.4 weeks.

In [ ]:
!pip3 install -q ultralytics

In [ ]:
from ultralytics import YOLO
import os
import yaml

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# data_choice = input("Which dataset do you want to download? Enter 'v1', 'v2', or 'v3': ")

# # Download and unzip the specified data version
# if data_choice.lower() == 'v1':
#     print("\nDownloading and unzipping v1...")
#     !gdown --fuzzy https://drive.google.com/file/d/10XGskgMGdkXTqbGDGu9BRtJklq_lfFTd/view?usp=sharing
#     !unzip -q data1*.zip
# elif data_choice.lower() == 'v2':
#     print("\nDownloading and unzipping v2...")
#     !gdown --fuzzy https://drive.google.com/file/d/1zXwvKLxzzOBp1bQ67nqfOgM55BI6sHho/view?usp=sharing
#     !unzip -q data2*.zip
# elif data_choice.lower() == 'v3':
#     print("\nDownloading and unzipping v3...")
#     !gdown --fuzzy https://drive.google.com/file/d/1K-jaOylXjukvCC6Uu8Ms2e7LxfL0-qWg/view?usp=sharing
#     !unzip -q data3*.zip
# else:
#     print("Invalid input.")


# Connecting to 'My Drive', to be able to access local data

from google.colab import drive
drive.mount('/content/drive')

# Download and unzip models if not found.
if not os.path.exists('yolov8s-seg.pt'):
    print('\nDownloading and unzipping YOLOv8 models...')
    !gdown --fuzzy https://drive.google.com/file/d/189U5fTomw3-QPjPpkppiXVSKa-sVIQIl/view?usp=sharing
    print('Unzipping models...')
    !unzip -q yolov8_-seg.zip
    print('Models downloaded')
else:
    print('\nModels already downloaded')

# Download the training arguments
print('\nDownloading training arguments...')
!gdown --fuzzy https://drive.google.com/file/d/13KLv4ywEVNVQHWhahIapSDDEzJWcAmT_/view?usp=sharing  # /content/args_500e.yaml
!gdown --fuzzy https://drive.google.com/file/d/1_N8H6akiKqFwezDjqfjVuXDvuP6zJZGP/view?usp=sharing  # /content/args_500e_tuned.yaml

Which dataset do you want to download? Enter 'v1', 'v2', or 'v3': v1
Mounted at /content/drive

Downloading...
From (original): https://drive.google.com/uc?id=189U5fTomw3-QPjPpkppiXVSKa-sVIQIl
From (redirected): https://drive.google.com/uc?id=189U5fTomw3-QPjPpkppiXVSKa-sVIQIl&confirm=t&uuid=8efab35b-4fa4-449e-a6b2-c3e18d6f93ca
To: /content/yolov8_-seg.zip
100% 298M/298M [00:05<00:00, 53.9MB/s]
Unzipping models...
Models downloaded

Downloading...
From: https://drive.google.com/uc?id=13KLv4ywEVNVQHWhahIapSDDEzJWcAmT_
To: /content/args_500e.yaml
100% 504/504 [00:00<00:00, 2.27MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_N8H6akiKqFwezDjqfjVuXDvuP6zJZGP
To: /content/args_500e_tuned.yaml
100% 516/516 [00:00<00:00, 2.92MB/s]


In [ ]:
os.getcwd()

'/content'

In [ ]:
# Write the YAML file for YOLO that contains the class names and train/val paths
YAML = 'data.yaml'

# # Set the path based on the version of the dataset
# if data_choice == 'v1':
#     path = f'{os.getcwd()}/ramp_data_yolo'
# elif data_choice == 'v2':
#     path = f'{os.getcwd()}/ramp_data_yolo_iou'
# elif data_choice == 'v3':
#     path = f'{os.getcwd()}/ramp_data_yolo_iou_scale'

path = f'{os.getcwd()}/drive/MyDrive/YOLO_test/data/ramp_data_yolo'

# Set the attribute for the YAML file
attr = {
    'path': path,
    'train': 'train/images',
    'val': 'val/images',
    'names': {
        0: 'rooftop'
    }
}

# Print the path
print(f'Writing the data file with path={path}')

# Write the file
with open(YAML, 'w') as f:
    yaml.dump(attr, f)

Writing the data file with path=/content/drive/MyDrive/YOLO_test/data/ramp_data_yolo


In [ ]:
# # HIGHLY RECOMMENDED if your training is of a long duration
# # Connect to Google Drive so that if session is terminated you can resume
# from google.colab import drive

# # Uncomment if needed
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# For testing purposes the fraction argument can be used to train on a subset of the data
FRACTION = 1.0

# Setting the starting model
MODEL = 'yolov8s-seg.pt'

# Setting the project name
PROJECT = '/content/drive/MyDrive/Omdena'

# Load the arguments from YAML
ARGS_FN = 'args_500e_tuned.yaml'
with open(ARGS_FN, 'r') as f:
    args = yaml.safe_load(f)

# Generate a name for the training session
name = f'{MODEL}_data_{data_choice}'
name = name.replace('.pt', '')

# Prompt the user for input
choice = input("Do you want to start or resume training? Enter 'new' for new training or 'resume' to resume training: ")

# Process the user input
if choice.lower() == 'new':
    print("\nStarting new training...\n")

    # Load the model
    model = YOLO(MODEL)

    # Train the model
    results = model.train(data=YAML, fraction=FRACTION, project=PROJECT, name=name, **args)

elif choice.lower() == 'resume':
    print("Resuming training...")

    # Load the model
    modelwpath = f'{PROJECT}/{name}/weights/last.pt'
    print(f'Loading model from {modelwpath}')
    model = YOLO(modelwpath)

    # Resume training
    results = model.train(resume=True)

else:
    print("Invalid input.")

Do you want to start or resume training? Enter 'new' for new training or 'resume' to resume training: new

Starting new training...

Ultralytics 8.3.4 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=data.yaml, epochs=50, time=None, patience=500, batch=64, imgsz=256, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Omdena, name=yolov8s-seg_data_v13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=False, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=Non

train: Scanning /content/drive/MyDrive/YOLO_test/data/ramp_data_yolo/train/labels.cache... 379 images, 93 backgrounds, 0 corrupt: 100%|██████████| 379/379 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/YOLO_test/data/ramp_data_yolo/train/images/OAM-180416-316406-19.jpg: 6 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/YOLO_test/data/ramp_data_yolo/train/images/OAM-180416-316407-19.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/YOLO_test/data/ramp_data_yolo/train/images/OAM-180417-316406-19.jpg: 5 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/YOLO_test/data/ramp_data_yolo/train/images/OAM-180417-316407-19.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/YOLO_test/data/ramp_data_yolo/train/images/OAM-180418-316406-19.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/YOLO_test/data/ramp_data_yolo/train/images/OAM-180418-316407-19.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/YOLO_test/data/ramp_data_yolo/train/images/OAM-180419-316406-19.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.1GB RAM): 100%|██████████| 379/379 [00:02<00:00, 171.54it/s]

...

                   all        258        224          0          0          0          0          0          0          0          0



50 epochs completed in 0.059 hours.
Optimizer stripped from /content/drive/MyDrive/Omdena/yolov8s-seg_data_v13/weights/last.pt, 21.2MB
Optimizer stripped from /content/drive/MyDrive/Omdena/yolov8s-seg_data_v13/weights/best.pt, 21.2MB

Validating /content/drive/MyDrive/Omdena/yolov8s-seg_data_v13/weights/best.pt...
Ultralytics 8.3.4 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-seg summary (fused): 213 layers, 10,482,067 parameters, 0 gradients, 37.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.83it/s]

                   all        258        224          0          0          0          0          0          0          0          0


Speed: 0.0ms preprocess, 1.2ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /content/drive/MyDrive/Omdena/yolov8s-seg_data_v13
